# Neuron, Astro, Not Cellを選択するGUI

In [1]:
# raw fluor, neuropil, deconvの波形からneuronを抽出
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
import tifffile
import cv2
import random
# from setGUIFont import *
import datetime
import time

from PyQt5.QtWidgets import *
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QColor, QFont, QPainterPath, QBrush
from PyQt5.QtCore import Qt, QTimer, QItemSelection
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

dir_notebook = os.path.dirname(os.path.abspath("__file__"))
# 親ディレクトリのパスを取得
dir_parent = os.path.dirname(dir_notebook)
if not dir_parent in sys.path:
    sys.path.append(dir_parent)

print(dir_parent)

from optic.gui import *
from optic.io import *
from optic.utils import *
from optic.preprocessing import *

class Suite2pROICheckGUI(QMainWindow, WidgetManager):
    def __init__(self):
        """
        pri: 格納するkey
        """
        QMainWindow.__init__(self)
        WidgetManager.__init__(self)
        self.setWindowTitle("Suite2pROICheckGUI")
        self.setGeometry(100, 100, 1200, 200)
        self.setupUI_done = False
        self.class_HoloStimResponse = None
        
        self.initializeAttributes()
    

        # Tableの列名と列番号, 列情報のdict
        self.dict_tablecol = {
            "Cell ID": {"order": 0, "type": "id", "editable": False},
            "Astrocyte": {"order": 1, "type": "radio", "group": "celltype"},
            "Neuron": {"order": 2, "type": "radio", "group": "celltype", "default": True},
            "Not Cell": {"order": 3, "type": "radio", "group": "celltype"},
            "Check": {"order": 4, "type": "checkbox"},
            "Tracking": {"order": 5, "type": "checkbox"},
            "Memo": {"order": 6, "type": "string", "editable": True}
        }
        # キーと機能のマッピング
        self.key_function_map = {
            Qt.Key_Z: ('radio', 1),
            Qt.Key_X: ('radio', 2),
            Qt.Key_C: ('radio', 3),
            Qt.Key_V: ('checkbox', 4),
            Qt.Key_B: ('checkbox', 5),
            Qt.Key_U: ('move', 'cell_type', -1),
            Qt.Key_I: ('move', 'skip_checked', -1),
            Qt.Key_O: ('move', 'skip_unchecked', -1),
            Qt.Key_J: ('move', 'cell_type', 1),
            Qt.Key_K: ('move', 'skip_checked', 1),
            Qt.Key_L: ('move', 'skip_unchecked', 1),
            Qt.Key_H: ('move', 'selected_type', 1), 
            Qt.Key_Y: ('move', 'selected_type', -1),
        }
        
        self.setupFileLoadUI()
        
    # アトリビュート初期化
    def initializeAttributes(self):
        self.dict_opacity     = {}
        self.dict_roiColors   = {}
        self.dict_table_selectedRow = {}
        self.dict_eventfile   = {}
        
        self.dict_Fall          = {}
        self.dict_im            = {}
        self.dict_im_ref        = {}
        self.ROIOpacity_init    = 50 # ROI透明度の初期値
        # canvas用変数
        self.plot_range = {'pri': [0, None]}  # [start_time, end_time]
        self.dragging = False
        self.last_x = 0

        self.roi_data_cache    = {}
        self.time_axis_cache    = {}
        self.event_data_cache    = {}
        
        self.block_radio_signals = False
        
    # Widget, LayoutなどUI設定用の関数
    """
    UI setup Function
    """    
    # File load用のUIセットアップ
    def setupFileLoadUI(self):
        self.mainLayout = QGridLayout()
        
        # checkroi_fix, setup, checkroi_movのLayoutを配置するためのLayout
        self.Layout_setupcheckroi = QHBoxLayout()
        
        Layout_setup = QVBoxLayout()
        # ファイルパスとbrowseボタン
        layout_path_fall     = self.makeLayoutLoadFileWidget(label="Fall mat file path", key="path_fall_pri", filetype="mat")
        layout_path_reftif   = self.makeLayoutLoadFileWidget(label="Reference Tiff image file path (optional)", key="path_reftif_pri", filetype="tiff")
        layout_path_cellpose = self.makeLayoutLoadFileWidget(label="Cellpose Mask path (optional)", key="path_cellpose_pri", filetype="npy")

        self.Layout_setup_button = QHBoxLayout()
        # Load
        self.Layout_setup_button.addWidget(self.makeWidgetButton(key="loadFile", label="Load files", func_=self.loadFilePathsandInitialize))
        # Exitボタンの設定
        self.Layout_setup_button.addWidget(self.makeWidgetButton(key="exit", label="Exit", func_=self.exitApp))
        # Helpボタン
        self.Layout_setup_button.addWidget(self.makeWidgetButton(key="help", label="Help", func_=None))
        
        Layout_setup.addLayout(layout_path_fall)
        Layout_setup.addLayout(layout_path_reftif)
        Layout_setup.addLayout(layout_path_cellpose)
        Layout_setup.addLayout(self.Layout_setup_button)
        Layout_setup.setSpacing(0)  # レイアウト間のスペーシングを設定
        
        self.Layout_setupcheckroi.addLayout(Layout_setup)
        
        self.mainLayout.addLayout(self.Layout_setupcheckroi, 2, 0, 1, 3)
        
        widget = QWidget()
        widget.setLayout(self.mainLayout)
        self.setCentralWidget(widget)
        
    # UIセットアップ
    def setupUI(self):
        # 新たに追加する3つのレイアウト
        layout_upper_left = self.makeLayoutLeftUpper()
        layout_upper_middle = self.makeLayoutMiddleUpper()
        layout_upper_right = self.makeLayoutRightUpper()
        
        self.mainLayout.addLayout(layout_upper_left, 1, 0, 1, 1)
        self.mainLayout.addLayout(layout_upper_middle, 1, 1, 1, 1)
        self.mainLayout.addLayout(layout_upper_right, 1, 2, 1, 1)
    
    """
    makeLayoutであるが実質makeWidgetとして機能する関数
    """
    # QLineEdit Layout label付き
    def makeLayoutLineEditLabel(self, key_label, key_lineedit, label, text_set="", width_fix=None):
        layout = QVBoxLayout()
        layout.addWidget(self.makeWidgetLabel(key=key_label, label=label))
        layout.addWidget(self.makeWidgetLineEdit(key=key_lineedit, text_set=text_set, width_fix=width_fix))
        return layout
    
    # QButtonGroup Layout
    def makeLayoutButtonGroup(self, key, list_label, set_exclusive=True):
        layout = QHBoxLayout()
        self.dict_buttongroup[key] = QButtonGroup(self)
        self.dict_buttongroup[key].setExclusive(set_exclusive)

        for i, label in enumerate(list_label):
            radioButton = QRadioButton(label)
            if i == 0:  # 1番目にチェック
                radioButton.setChecked(True)
            layout.addWidget(radioButton)
            self.dict_buttongroup[key].addButton(radioButton, i)

        return layout
    
    # QSlider Layout label付き
    def makeLayoutSliderLabel(self, key_label, key_slider, label, align=Qt.AlignLeft, func_=None, value_min=0, value_max=255, value_set=10, height=10, axis=Qt.Horizontal):
        layout = QVBoxLayout()
        label = self.makeWidgetLabel(key_label, label)
        slider = self.makeWidgetSlider(key_slider, func_, value_min, value_max, value_set, height, axis)
        layout.addWidget(label)
        layout.addWidget(slider)
        return layout
        
    # Layout作成関数
    """
    make Layout Function
    """
    # 読み込むファイルを選択するためのウィジェット
    def makeLayoutLoadFileWidget(self, label="", key="", filetype=None):
        layout = QHBoxLayout() # entry
        layout.addLayout(self.makeLayoutLineEditLabel(key_label=key, key_lineedit=key, label=label))
        layout.addWidget(self.makeWidgetButton(key=key, label="Browse", func_=lambda: openFileDialogAndSetLineEdit(self, filetype, self.dict_lineedit[key])))
        return layout
    
    # Eventfileを重ねてプロット, Eventfile前後のプロット用Layout
    def makeLayoutEventAlignment(self):
        layout = QHBoxLayout()
        
        # Eventの前後何フレーム分プロットするか
        layout_plotrange = QVBoxLayout()
        # EventFileにalignしたtraceのプロットコンフィグ
        layout_plotrange.addLayout(self.makeLayoutLineEditLabel(key_label="eventfile_align_plot_range",
                                                               key_lineedit="eventfile_align_plot_range",
                                                               label="plot range from Event start (pre, post; sec)",
                                                               text_set="(10, 10)"))
        # プロットに重ねるEventFile npyファイルの読み込みボタン, Clearボタン
        layout_plotrange.addWidget(self.makeWidgetButton(key="loadEventFile", label="Load EventFile npy file", func_=lambda : self.loadEventFile(key="pri")))         
        layout_plotrange.addWidget(self.makeWidgetButton(key="clearEventFile", label="Clear", func_=lambda : self.clearEventFile(key="pri")))
                                   
        # zoom時にどこまで拡大するか, 読み込んだEventFileのtraceをプロットするか
        layout_zoomrangecheck = QVBoxLayout()
        layout_zoomrangecheck.addLayout(self.makeLayoutLineEditLabel(key_label="minPlotRange",
                                                               key_lineedit="minPlotRange",
                                                               label="Minimum plot range (sec)",
                                                               text_set="30"))
        layout_zoomrangecheck.addWidget(self.makeWidgetCheckBox(key="plot_eventfile_trace", 
                                                                label="plot EventFile trace", 
                                                                checked=True,
                                                                func_=lambda: self.updatePlot("pri")))
        """
        HfixHoloCS用 HoloStimのresponseを求める
        """
        layout_zoomrangecheck.addWidget(self.makeWidgetButton(key="holostim_response",
                                                              label="Calculate HoloStim Response",
                                                              func_=self.runHoloStimResponse))

        layout.addLayout(layout_plotrange)
        layout.addLayout(layout_zoomrangecheck)
        return layout
        
    # 選択したROIのstatを表示するLayout
    def makeLayoutLabelROIStat(self):
        layout = QVBoxLayout()
        # ROIのsize, radius, aspect_ratio, compact, footprint, skew(歪度), std
        for list_label_key in (["npix", "radius", "aspect_ratio", "compact", "footprint"], ["skew", "std"]):
            layout_hbox = QHBoxLayout()
            for label_key in list_label_key:
                layout_hbox.addWidget(self.makeWidgetLabel(f"display_{label_key}", f"{label_key}: "))
            layout.addLayout(layout_hbox)
        return layout

    # ROIの表示を変更するボタン用Layout
    def makeLayoutButtonROIShow(self):
        layout = QVBoxLayout()
        
        # ROIの表示切り替え All, Cell, Not Cell
        roidisp_options = ["All ROI"]
        roidisp_options.extend([key for key, value in self.dict_tablecol.items() if value['type'] == 'radio'])
        roidisp_options.append("None")
        layout_roidisp = self.makeLayoutButtonGroup("roidisp", roidisp_options)
        self.dict_buttongroup["roidisp"].buttonClicked.connect(lambda button: self.onROIDispButtonGroupsChanged("pri", button.text())) # 関数紐づけ
        # ROIの表示切り替え stat threshold
        layout_roidisp_stat = QHBoxLayout()
        layout_roidisp_stat.addWidget(self.makeWidgetCheckBox(key="roidisp_stat", label="ROI Show Threshold"))
        for label in ["npix", "compact"]:
            layout_roidisp_stat.addLayout(self.makeLayoutLineEditLabel(key_label=f"roidisp_{label}", 
                                                                       key_lineedit=f"roidisp_{label}",
                                                                       label=label))
        
        
        # 背景画像の切り替え meanImg, meanImgE, max_proj, Vcorr, RefImg
        layout_refimg = self.makeLayoutButtonGroup("refimg", ["meanImg", "meanImgE", "max_proj", "Vcorr"])
        self.dict_buttongroup["refimg"].buttonClicked.connect(lambda button: self.onRefImgButtonGroupChanged("pri", button.text())) # 関数紐づけ
        
        # 画面クリック時 Astrocyte, Neuron, Not Cell, Check, TrackingのROIをスキップするか
        layout_skiproi = QHBoxLayout()
        skip_items = [key for key, value in self.dict_tablecol.items() if value['type'] in ['radio', 'checkbox']]
        for item in skip_items:
            checkbox_key = f"skip_{item}"
            checkbox_label = f"Skip {item} ROI"
            layout_skiproi.addWidget(self.makeWidgetCheckBox(checkbox_key, checkbox_label))
            
        # Move ROIs Mask Imageチェックボックスの設定
        layout_roimove = QHBoxLayout()
        layout_roimove.addWidget(self.makeWidgetCheckBox("moveROIImage", "Move ROI Mask Image"))

        layout.addLayout(layout_roidisp)
        layout.addLayout(layout_roidisp_stat)
        layout.addLayout(layout_refimg)
        layout.addLayout(layout_skiproi)
        layout.addLayout(layout_roimove)
        return layout
    
    # 画像のコントラスト調節スライダー、表示チェックボックス用Layout
    def makeLayoutContrastSlider(self, key, label_checkbox, label_label, func_slider=None, func_checkbox=None):
        layout = QVBoxLayout()
        # チェックボックスの設定
        layout.addWidget(self.makeWidgetCheckBox(key, label_checkbox, func_=func_checkbox, checked=True))

        # Min, Max Valueスライダーの設定
        for m, value_set in zip(["min", "max"], [0, 255]):
            layout.addLayout(self.makeLayoutSliderLabel(key_label=f"value_{key}_{m}", 
                                                        key_slider=f"value_{key}_{m}", 
                                                        label=f"{m} {label_label}", 
                                                        value_set=value_set, 
                                                        func_=func_slider))
        return layout
    
    # 透明度調節スライダーのLayout
    def makeLayoutOpacitySlider(self):
        layout = QHBoxLayout()
        for (key, label, default_value) in zip(['opacity_allroi', 'opacity_selectedroi'],
                                               ['Opacity of All ROI', 'Opacity of Selected ROI'],
                                                [50, 255]):
            layout.addLayout(self.makeLayoutSliderLabel(key_label=key, 
                                                        key_slider=key, 
                                                        label=label, 
                                                        value_set=default_value,
                                                        func_=lambda value, k="pri": self.onSliderValueChanged(k)))
        return layout
    
    # スライダーをまとめたLayout
    def makeLayoutAllSlider(self):
        layout = QGridLayout()
        for i, channel in enumerate(["pri", "sec"]):
            layout.addLayout(self.makeLayoutContrastSlider(key=f"{channel}", 
                                                           label_checkbox=f"Show {channel}", 
                                                           label_label=f"Value ({channel})",
                                                           func_checkbox=lambda state, key="pri", channel=channel: self.toggleChannelVisibility(key, state, channel),
                                                           func_slider=lambda value, k="pri": self.onSliderValueChanged(k)), 
                                                           1, i+1, 1, 1)
        layout.addLayout(self.makeLayoutOpacitySlider(), 2, 1, 1, 2)
        return layout
    

    # celltypeのROI数を表示するLayout
    def makeLayoutLabelROINumber(self):
        layout = QHBoxLayout()
        # ROI number label
        list_celltype = [key for key in self.dict_tablecol.keys() if self.dict_tablecol[key]["type"] == "radio"] + ["All"]
        text = ""
        for celltype in list_celltype:
            text += f"{celltype}: 0, "
        layout.addWidget(self.makeWidgetLabel(key="ROInumber", label=text))  # レイアウトにラベルを追加
        return layout
    
    # ROI全部をそろえる, ROICheckを保存、読み込むボタン用のLayout
    def makeLayoutButtonROICheck(self):
        layout = QVBoxLayout()
        # All ROI set button, QHBoxLayoutにまとめる
        layout_roicheck = QHBoxLayout()

        # dict_tablecol からラジオボタンタイプのカラムを抽出
        radio_columns = [(key, value) for key, value in self.dict_tablecol.items() if value['type'] == 'radio']

        for col_name, col_info in radio_columns:
            key = f"set_{col_name}"
            label = f"Set {col_name}"
            layout_roicheck.addWidget(self.makeWidgetButton(key=key, label=label, 
                                                            func_=lambda checked, cn=col_name: self.setAllROISameType("pri", cn)))
        # ROICheck save, loadボタン
        layout_roiIO = QHBoxLayout()
        for key, label, func_ in zip(["saveROICheck", "loadROICheck", "loadROIMatch"], 
                                      ["Save ROICheck", "Load ROICheck", "Load ROI Match"], 
                                      [lambda: self.saveROICheck(key="pri"), lambda: self.loadROICheck(key="pri"), None]):
            layout_roiIO.addWidget(self.makeWidgetButton(key=key, label=label, func_=func_))
        layout.addLayout(layout_roicheck)
        layout.addLayout(layout_roiIO)
        return layout
    
    # Noise判定用のROIstatのthreshold
    def makeLayoutROIFliterThreshold(self):
        layout = QGridLayout()
        # Cell Filter 2x3 boxlayout
        for i, (key, text_set) in enumerate(zip(["npix", "radius", "aspect_ratio", "compact", "skew", "std"], 
                                                ["(50, 200)", "(3, 12)", "(0, 1.5)", "(0, 1.5)", "(1, 100)", "(0, 100)"])):
            layout.addLayout(self.makeLayoutLineEditLabel(key_label=f"threshold_{key}", key_lineedit=f"threshold_{key}", label=key, text_set=text_set, width_fix=100), i//3, i%3, 1, 1)
        
        layout.addWidget(self.makeWidgetLabel(key="threshold_params", label="<- thresholds (min, max)"), 0, 3, 1, 1)
        layout.addWidget(self.makeWidgetButton(key="filterROI", label="Filter ROI", func_=lambda : self.filterROI(key="pri")), 1, 3, 1, 1)
        return layout
    
    # 画像表示用のview Layout
    def makeLayoutViewROICheck(self, key, func_click=None, width_min=530, height_min=530, color="black"):
        layout = QVBoxLayout()
        
        # ROI表示画像
        self.dict_scene[key] = QGraphicsScene()
        self.dict_view[key] = QGraphicsView(self.dict_scene[key])
        self.dict_view[key].setRenderHint(QPainter.Antialiasing)  # アンチエイリアシングを有効化
        self.dict_view[key].setRenderHint(QPainter.SmoothPixmapTransform)  # スムーズなピクセルマップ変換を有効化
        self.dict_view[key].setMinimumHeight(width_min)
        self.dict_view[key].setMinimumWidth(height_min)
        self.dict_view[key].setStyleSheet("background-color: black;")  # 背景色を黒に設定
        self.dict_view[key].setViewportUpdateMode(QGraphicsView.FullViewportUpdate)
        # クリック時のイベント
        if func_click:
            self.dict_view[key].mousePressEvent = func_click
        
        layout.addWidget(self.dict_view[key])
        return layout
    
    # traceプロット用のcanvas Layout
    def makeLayoutCanvasTracePlot(self, key):
        # プロットウィジェットの設定
        layout = QVBoxLayout()
        self.dict_figure[key] = Figure()
        self.dict_canvas[key] = FigureCanvas(self.dict_figure[key])
        for i, key_ax in enumerate(["top", "middle", "bottom"]):
            self.dict_ax[f"{key}_{key_ax}"] = self.dict_figure[key].add_subplot(3, 1, i+1)
        self.dict_figure[key].subplots_adjust(top=0.95, bottom=0.1, right=0.9, left=0.1, hspace=0.4)
        # イベントハンドラの設定
        self.dict_canvas[key].mpl_connect('scroll_event', lambda event: self.onScrollTopAxis(event, key))
        self.dict_canvas[key].mpl_connect('button_press_event', lambda event: self.onPressTopAxis(event, key))
        self.dict_canvas[key].mpl_connect('button_release_event', lambda event: self.onReleaseTopAxis(event, key))
        self.dict_canvas[key].mpl_connect('motion_notify_event', lambda event: self.onMotionTopAxis(event, key))
        self.dict_canvas[key].mpl_connect('button_press_event', lambda event: self.onClickMiddleAxis(event, key))
    
        layout.addWidget(self.dict_canvas[key])
        return layout
    
    # ROICheck用のTable Layout
    def makeLayoutTableROICheck(self, key):
        layout = QVBoxLayout()
        self.dict_table[key] = QTableWidget()
        self.dict_table[key] = self.setupTableWidget(key)
        layout.addWidget(self.dict_table[key])
        return layout
    
    """
    make Layout Function; 区画単位で指定
    """
    # 左上
    def makeLayoutLeftUpper(self):
        layout = QVBoxLayout()
        layout_canvas = self.makeLayoutCanvasTracePlot(key="pri")
        layout_plotconfig = self.makeLayoutEventAlignment()
        layout.addLayout(layout_canvas, stretch=1) # stretch; 余った領域を自動的に活用
        layout.addLayout(layout_plotconfig)
        return layout
    
    # 中上
    def makeLayoutMiddleUpper(self):
        layout = QVBoxLayout()
        layout_view = self.makeLayoutViewROICheck(key="pri", func_click=lambda event, k="pri": self.onViewClicked(event, key=k))
        layout_roistat = self.makeLayoutLabelROIStat()
        layout_buttongroup = self.makeLayoutButtonROIShow()
        layout_slider = self.makeLayoutAllSlider()
        
        layout.addLayout(layout_view, stretch=1)
        for layout_ in [layout_roistat, layout_buttongroup, layout_slider]:
            layout.addLayout(layout_)
        return layout
    
    # 右上
    def makeLayoutRightUpper(self):
        layout = QVBoxLayout()
        layout_table = self.makeLayoutTableROICheck(key="pri")
        layout_roinum = self.makeLayoutLabelROINumber()
        layout_button = self.makeLayoutButtonROICheck()
        layout_threshold = self.makeLayoutROIFliterThreshold()
        
        layout.addLayout(layout_table, stretch=1)
        for layout_ in [layout_roinum, layout_button, layout_threshold]:
            layout.addLayout(layout_)
        return layout

    # データ入力、出力用関数
    """
    IO Function
    """
            
    # 指定したファイルパスを読み込みして初期化
    def loadFilePathsandInitialize(self):
        self.initialize_gui = False
        print("Files Loading...")
        
        # data読み込み
        self.initializeGUI()
        self.initializeROIColors()
        # UI初期化
        if not self.setupUI_done:
            self.setupUI()
            self.setupUI_done = True
        else:
            self.initializeUI(key="pri")
            
        # view, table初期化
        self.dict_table["pri"] = self.setupTableWidget(key="pri")
        self.displayROINumber("pri")

        print("GUI Initializing...")
        self.initialize_gui = True
        
    # GUI, データの初期化
    def initializeGUI(self):
        # データの完全なリセット
        self.initializeAttributes()
        
        key = "pri"
        Fall = loadmat(self.dict_lineedit[f"path_fall_{key}"].text())
        self.dict_Fall[key] = convertMatToDictFall(Fall)
        
        self.dict_table_selectedRow[key] = 0
        # eventfile初期化
        self.dict_eventfile[key] = None
        # dict_imに画像を保存
        self.dict_im[key] = {}
        # まず、meanImgのサイズを基準サイズとして取得
        base_shape = self.dict_Fall[key]["ops"]["meanImg"].shape
        for img_key in ["meanImg", "meanImgE", "max_proj", "Vcorr"]:
            img = convertImageDtypeToINT(self.dict_Fall[key]["ops"][img_key], dtype="uint8")
            img = resizeImageShape(img, base_shape)
            self.dict_im[key][img_key] = img
            
        if self.class_HoloStimResponse:
            self.class_HoloStimResponse.close()
            
    def initializeUI(self, key):
        # UIの各要素を新しいデータに合わせて更新
        for ax_key in self.dict_ax:
            self.dict_ax[ax_key].clear()
        self.dict_table[key].clearContents()
        self.dict_table[key].setRowCount(0)
        self.dict_scene[key].clear()
        




    # EventFileの読み込み
    def loadEventFile(self, key):
        options = QFileDialog.Options()
        # npyを連結できるように複数ファイルを選択可能
        paths_eventfile, _ = QFileDialog.getOpenFileNames(self, "Open File", "", "npy Files (*.npy);;All Files (*)", options=options)
        if paths_eventfile:
            # 複数読み込んで連結
            self.dict_eventfile[key] = np.concatenate([np.load(path_eventfile) for path_eventfile in paths_eventfile])
            self.precalculate_event_data(key)
        self.updatePlot(key)
        
    # イベントデータの事前計算
    def precalculate_event_data(self, key):
        self.event_data_cache = {}
        F_data = self.dict_Fall[key]["F"]
        event_data = self.dict_eventfile[key]

        # ROI IDのリストを取得し、整数に変換
        roi_ids = list(F_data.keys() if isinstance(F_data, dict) else range(len(F_data)))

        for roi_id in roi_ids:
            # F_dataが辞書の場合はキーでアクセス、そうでない場合はインデックスでアクセス
            f_data = F_data[roi_id] if isinstance(F_data, dict) else F_data[int(roi_id)]
            self.event_data_cache[roi_id] = self.scaleEventData(event_data, f_data)
            
    # Eventfile消去
    def clearEventFile(self, key):
        self.dict_eventfile[key] = None
        self.updatePlot(key)
        
    # Tableの内容をROICheckとして保存
    def saveROICheck(self, key):
        options = QFileDialog.Options()
        path_Fall = self.dict_lineedit[f"path_fall_{key}"].text()
        dir_project = os.path.dirname(path_Fall)
        name_Fall = os.path.basename(path_Fall).replace('Fall_', '')
        path_roicheck_init = os.path.join(dir_project, f"ROIcheck_{name_Fall}")
        path_roicheck, _ = QFileDialog.getSaveFileName(self, "Save ROI Check", path_roicheck_init, "mat Files (*.mat);;All Files (*)", options=options)
        if path_roicheck:
            try:
                tableWidget = self.dict_table[key]
                today = datetime.datetime.today().strftime('%y%m%d')

                cell_type_keys = {
                    "Neuron": "rows_selected_neuron",
                    "Astrocyte": "rows_selected_astro",
                    "Not Cell": "rows_selected_noise"
                }

                data_to_save = {}
                roi_count = tableWidget.rowCount()

                for col_name, col_info in self.dict_tablecol.items():
                    if col_info['type'] == 'radio':
                        selected_rows = []
                        for row in range(roi_count):
                            radio_button = tableWidget.cellWidget(row, col_info['order'])
                            if radio_button and radio_button.isChecked():
                                selected_rows.append([row])

                        data_to_save[col_name] = np.array(selected_rows, dtype=np.int32)
                        if col_name in cell_type_keys:
                            data_to_save[cell_type_keys[col_name]] = np.array(selected_rows, dtype=np.int32)
                    elif col_info['type'] == 'checkbox':
                        data_to_save[col_name] = np.zeros((roi_count, 1), dtype=np.bool_)
                        for row in range(roi_count):
                            item = tableWidget.item(row, col_info['order'])
                            data_to_save[col_name][row] = item.checkState() == Qt.Checked if item else False
                    elif col_info['type'] == 'string':
                        data_to_save[col_name] = np.empty((roi_count, 1), dtype=object)
                        for row in range(roi_count):
                            item = tableWidget.item(row, col_info['order'])
                            data_to_save[col_name][row] = item.text() if item else ''

                threshold_roi = {param: self.dict_lineedit[f"threshold_{param}"].text() for param in ["npix", "radius", "aspect_ratio", "compact", "skew", "std"]}
                
                mat_roicheck = {
                    "manualROIcheck": {
                        **data_to_save,
                        "update": today,
                        "threshold_roi": threshold_roi,
                    }
                }
                savemat(path_roicheck, mat_roicheck)
                print("ROICheck file saved!")
            except Exception as e:
                print(f"Error saving ROICheck file: {e}")
    # ROICheckを読み込んでTableを更新
    def loadROICheck(self, key):
        options = QFileDialog.Options()
        path_Fall = self.dict_lineedit[f"path_fall_{key}"].text()
        dir_project = os.path.dirname(path_Fall)
        path_roicheck = openFileDialog(self, file_type="mat", title="Open Fall.mat File", initial_dir=dir_project)
        if path_roicheck:
            try:
                mat_roicheck = loadmat(path_roicheck)
                mat_roicheck = mat_roicheck["manualROIcheck"]
                dict_roicheck = convertMatToDictROICheck(mat_roicheck)

                tableWidget = self.dict_table[key]
                roi_count = tableWidget.rowCount()

                cell_type_keys = {
                    "rows_selected_neuron": "Neuron",
                    "rows_selected_astro": "Astrocyte",
                    "rows_selected_noise": "Not Cell"
                }

                # ラジオボタンの設定
                for col_name, col_info in self.dict_tablecol.items():
                    if col_info['type'] == 'radio':
                        if col_name in dict_roicheck:
                            selected_rows = dict_roicheck[col_name]
                            for row in range(roi_count):
                                radio_button = tableWidget.cellWidget(row, col_info['order'])
                                if radio_button:
                                    radio_button.setChecked(any(row == sr[0] for sr in selected_rows))
                        elif col_name in cell_type_keys.values():
                            corresponding_key = [k for k, v in cell_type_keys.items() if v == col_name][0]
                            if corresponding_key in dict_roicheck:
                                selected_rows = dict_roicheck[corresponding_key]
                                for row in range(roi_count):
                                    radio_button = tableWidget.cellWidget(row, col_info['order'])
                                    if radio_button:
                                        radio_button.setChecked(any(row == sr[0] for sr in selected_rows))

                # チェックボックスと文字列の設定
                for col_name, col_info in self.dict_tablecol.items():
                    if col_info['type'] in ['checkbox', 'string']:
                        if col_name in dict_roicheck:
                            data = dict_roicheck[col_name]
                            for row in range(min(roi_count, len(data))):
                                item = tableWidget.item(row, col_info['order'])
                                if item:
                                    if col_info['type'] == 'checkbox':
                                        item.setCheckState(Qt.Checked if data[row][0] else Qt.Unchecked)
                                    else:  # string
                                        # 空のリストや空の文字列を空白として処理
                                        value = str(data[row][0])
                                        if value == '[]' or value == '':
                                            value = ''
                                        item.setText(value)

                # threshold_roi の設定
                if 'threshold_roi' in dict_roicheck:
                    threshold_roi_dict = convertMatToDictFall(dict_roicheck["threshold_roi"])
                    for param in ["npix", "radius", "aspect_ratio", "compact", "skew", "std"]:
                        if param in threshold_roi_dict:
                            self.dict_lineedit[f"threshold_{param}"].setText(str(threshold_roi_dict[param][0]))

                self.updateView(key)
                self.displayROINumber(key)
                print("ROICheck file loaded!")
            except Exception as e:
                print(f"Error loading ROICheck file: {e}")
                raise  # This will re-raise the exception for debugging purposes

            
    """
    Table Widget Function
    """
    # TableWidgetのsetup
    def setupTableWidget(self, key): # key: pri
        tableWidget = self.dict_table[key]
        tableWidget.clearSelection() # テーブルの選択初期化
        list_cellid = list(self.dict_Fall[key]["stat"].keys())
        tableWidget.setRowCount(len(list_cellid))
        # 列を順序に基づいてソート
        sorted_columns = sorted(self.dict_tablecol.items(), key=lambda x: x[1]['order'])
        tableWidget.setColumnCount(len(sorted_columns))

        # ヘッダーの設定
        tableWidget.setHorizontalHeaderLabels([col[0] for col in sorted_columns])
        # テーブルの選択モードを単一行選択に設定
        tableWidget.setSelectionMode(QAbstractItemView.SingleSelection)

        # 1行ずつ指定
        for cellid in list_cellid:
            for col_name, col_info in sorted_columns:
                col_index = col_info['order'] # Cell ID
                # Cell ID
                if col_info["type"] == "id":
                    cellItem = QTableWidgetItem(f"{cellid}")
                    cellItem.setFlags(cellItem.flags() & ~Qt.ItemIsEditable)
                    tableWidget.setItem(cellid, col_index, cellItem)
                # radiobutton
                elif col_info["type"] == "radio":
                    radioButton = QRadioButton()
                    if col_info.get("default", False):
                        radioButton.setChecked(True)
                    tableWidget.setCellWidget(cellid, col_index, radioButton)
                # checkbox
                elif col_info["type"] == "checkbox":
                    checkBox = QTableWidgetItem()
                    checkBox.setCheckState(Qt.Unchecked)
                    tableWidget.setItem(cellid, col_index, checkBox)
                # String
                elif col_info["type"] == "string":
                    stringItem = QTableWidgetItem()
                    # 編集可能か
                    if not col_info.get("editable", True):
                        stringItem.setFlags(stringItem.flags() & ~Qt.ItemIsEditable)
                    tableWidget.setItem(cellid, col_index, stringItem)

        # ラジオボタンのグループ化
        self.groupRadioButtons(tableWidget)
        # ラジオボタンの状態変更を監視
        for row in range(tableWidget.rowCount()):
            for col_name, col_info in self.dict_tablecol.items():
                if col_info['type'] == 'radio':
                    radio_button = tableWidget.cellWidget(row, col_info['order'])
                    if radio_button:
                        radio_button.toggled.connect(lambda checked, r=row, c=col_name, k=key: self.onRadioButtonChanged(k, r, c, checked))

        # セルの横幅指定
        for i in range(tableWidget.columnCount()):
            tableWidget.setColumnWidth(i, 80)

        # イベント設定
        # 行の選択を変更したとき
        tableWidget.selectionModel().selectionChanged.connect(lambda selected, deselected: self.onTableSelectionChanged(key, selected, deselected))
        # キーボード操作
        tableWidget.keyPressEvent = lambda event: self.tableWidgetKeyPressEvent(event, key)

        return tableWidget
    
    # ラジオボタンのグループ化
    def groupRadioButtons(self, tableWidget):
        groups = {}
        for col_name, col_info in self.dict_tablecol.items():
            if col_info["type"] == "radio":
                group_name = col_info.get("group", "default")
                if group_name not in groups:
                    groups[group_name] = []
                groups[group_name].append(col_info['order'])

        for row in range(tableWidget.rowCount()):
            for group_name, columns in groups.items():
                buttonGroup = QButtonGroup(tableWidget)
                for col in columns:
                    radioButton = tableWidget.cellWidget(row, col)
                    if radioButton:
                        buttonGroup.addButton(radioButton)
                        
    # Tableのセルを選択したときの関数
    def onTableSelectionChanged(self, key, selected, deselected):
        if selected.indexes():
            roi_id = selected.indexes()[0].row()
            self.selectROI(key, roi_id)
            
    # ラジオボタンが変更されたときのハンドラ
    def onRadioButtonChanged(self, key, row, column_name, checked):
        # setAllROISameTypeなどで毎回呼び出されるのを防ぐ
        if self.block_radio_signals:
            return

        if checked:
            self.dict_table[key].selectRow(row)
            self.dict_table_selectedRow[key] = row
            self.displayROINumber(key)
            
    # すべてのROIを同じタイプに設定する関数
    def setAllROISameType(self, key, col_name):
        table = self.dict_table[key]
        col_index = self.dict_tablecol[col_name]['order']

    #     # テーブルの更新を一時的に無効化
        table.setUpdatesEnabled(False)
        # ラジオボタンの変更を一時的にブロック
        self.block_radio_signals = True
        for row in range(table.rowCount()):
            for column, col_info in self.dict_tablecol.items():
                if col_info['type'] == 'radio':
                    radio_button = table.cellWidget(row, col_info['order'])
                    if radio_button:
                        radio_button.setChecked(column == col_name)

        # ラジオボタンの変更のブロックを解除
        self.block_radio_signals = False

    #     # テーブルの更新を再開
        table.setUpdatesEnabled(True)

        # ROI数の表示を更新
        self.displayROINumber(key)
        # 現在選択されているROIのビューを更新
        if self.dict_table_selectedRow[key] is not None:
            self.selectROI(key, self.dict_table_selectedRow[key])
        # ビューを一括更新
        self.updateView(key)
            
    # statが閾値外にあるROIをNot Cellとして判定する
    def filterROI(self, key):
        reply = QMessageBox.question(self, 'Filter Cells', 'Filter ROIs?', QMessageBox.Yes | QMessageBox.No, QMessageBox.No)
        if reply == QMessageBox.Yes:
            parameters = ["npix", "radius", "aspect_ratio", "compact", "skew", "std"]

            # ラジオボタンの変更を一時的にブロック
            self.block_radio_signals = True
            for roi_id in self.dict_Fall[key]["stat"].keys():
                is_within_threshold = True
                for param in parameters:
                    threshold_str = self.dict_lineedit[f"threshold_{param}"].text()
                    threshold_min, threshold_max = map(float, threshold_str.strip("()").split(","))

                    value = self.dict_Fall[key]["stat"][roi_id][param].flatten()[0]

                    if not (threshold_min <= value <= threshold_max):
                        is_within_threshold = False
                        break

                if not is_within_threshold:
                    self.updateROIClassification(key, roi_id, "Not Cell")
                    
            # ラジオボタンの変更のブロックを解除
            self.block_radio_signals = False

            self.updateView(key)
            self.displayROINumber(key)

    def updateROIClassification(self, key, roi_id, classification):
        for col_name, col_info in self.dict_tablecol.items():
            if col_info['type'] == 'radio':
                radio_button = self.dict_table[key].cellWidget(roi_id, col_info['order'])
                if radio_button:
                    radio_button.setChecked(col_name == classification)
        

    # キーイベントのカスタム
    """
    keyPressEvent
    """
    def tableWidgetKeyPressEvent(self, event, key):
        """
        テーブルウィジェットのキーイベントを処理します。
        特定のキーに応じて、セルの選択や行の移動を行います。
        
        :param event: キーイベント
        :param key: テーブルの識別子
        """
        if self.dict_table_selectedRow[key] is not None:
            currentRow = self.dict_table_selectedRow[key]
            
            if event.key() in self.key_function_map:
                action = self.key_function_map[event.key()]
                if action[0] == 'radio':
                    self.selectRadioButton(key, currentRow, action[1])
                elif action[0] == 'checkbox':
                    self.toggleCheckBox(key, currentRow, action[1])
                elif action[0] == 'move':
                    if action[1] == 'cell_type':
                        self.moveToSameCellType(key, currentRow, action[2])
                    elif action[1] == 'skip_checked':
                        self.moveSkippingChecked(key, currentRow, action[2], True)
                    elif action[1] == 'skip_unchecked':
                        self.moveSkippingChecked(key, currentRow, action[2], False)
                    elif action[1] == 'selected_type':
                        self.moveToNextROIOfSelectedType(key, currentRow, action[2])
                        
                # 操作後にROIを更新
                self.updateView(key)
                
                new_row = self.dict_table_selectedRow[key]
                self.dict_table[key].setCurrentCell(new_row, 0)
                self.dict_table[key].scrollToItem(self.dict_table[key].item(new_row, 0))

        # イベントを親クラスに渡す
        super(type(self.dict_table[key]), self.dict_table[key]).keyPressEvent(event)
        
    # 指定された行と列のラジオボタンを選択
    def selectRadioButton(self, key, row, column):
        radioButton = self.dict_table[key].cellWidget(row, column)
        if isinstance(radioButton, QRadioButton):
            radioButton.setChecked(True)
        
    # 指定された行と列のチェックボックスの状態を切り替える
    def toggleCheckBox(self, key, row, column):
        checkBoxItem = self.dict_table[key].item(row, column)
        if checkBoxItem:
            checkBoxItem.setCheckState(Qt.Unchecked if checkBoxItem.checkState() == Qt.Checked else Qt.Checked)

    # 現在の行と同じセルタイプの次の行に移動
    def moveToSameCellType(self, key, startRow, direction):
        currentCellType = self.getCurrentCellType(key, startRow)
        totalRows = self.dict_table[key].rowCount()
        newRow = startRow

        while True:
            newRow += direction
            if not (0 <= newRow < totalRows):
                break
            if self.getCurrentCellType(key, newRow) == currentCellType:
                self.selectRow(key, newRow)
                return
            
    # checkbox roidispで選択しているcelltypeでのみ移動
    def moveToNextROIOfSelectedType(self, key, startRow, direction):
        selected_type = self.dict_buttongroup["roidisp"].checkedButton().text()
        if selected_type == "All ROI" or selected_type == "None":
            return  # これらの場合は移動しない

        totalRows = self.dict_table[key].rowCount()
        newRow = startRow

        while True:
            newRow = (newRow + direction) % totalRows  # 循環させる
            if newRow == startRow:  # 一周して戻ってきたら終了
                break
            
            for col_name, col_info in self.dict_tablecol.items():
                if col_info['type'] == 'radio' and col_name == selected_type:
                    radio_button = self.dict_table[key].cellWidget(newRow, col_info['order'])
                    if radio_button and radio_button.isChecked():
                        self.selectRow(key, newRow)
                        return

    # チェック状態に基づいて行をスキップしながら移動, 
    # skip_checked=TrueならCheckedがつけられていたらスキップ
    # direction=1で下方向
    def moveSkippingChecked(self, key, startRow, direction, skip_checked):
        totalRows = self.dict_table[key].rowCount()
        newRow = startRow

        while True:
            newRow += direction
            if not (0 <= newRow < totalRows):
                break
            if self.isRowChecked(key, newRow) != skip_checked:
                self.selectRow(key, newRow)
                return
    
    # 指定された行を選択し、関連する状態を更新
    def selectRow(self, key, row):
        self.dict_table[key].selectRow(row)
        self.dict_table_selectedRow[key] = row

    # 指定された行の現在のセルタイプ（ラジオボタンの選択状態）を取得
    def getCurrentCellType(self, key, row):
        for col_name, col_info in self.dict_tablecol.items():
            if col_info['type'] == 'radio':
                radioButton = self.dict_table[key].cellWidget(row, col_info['order'])
                if radioButton and radioButton.isChecked():
                    return col_name
        return None

    # 指定された行がチェックされているかどうかを確認
    def isRowChecked(self, key, row):
        check_col = next((col_info['order'] for col_name, col_info in self.dict_tablecol.items() if col_name == 'Check'), None)
        if check_col is not None:
            checkBoxItem = self.dict_table[key].item(row, check_col)
            return checkBoxItem.checkState() == Qt.Checked if checkBoxItem else False
        return False
            
                    
    """
    Slider Widget Function
    """
    # Sliderのvalue変更
    def onSliderValueChanged(self, key):
        self.updateView(key)
    
    """
    View Widget Function
    """
    # ROI用の色を作成
    def makeROIColors(self, num_colors):
        colors = []
        for _ in range(num_colors):
            color = [random.randint(100, 255) for _ in range(3)]
            colors.append(color)
        return colors
    
    # ROIの色初期化
    def initializeROIColors(self):
        self.dict_roiColors = {k:v for k,v in enumerate(self.makeROIColors(len(list(self.dict_Fall["pri"]["stat"].keys()))))}
    
    def updateView(self, key):
        if not hasattr(self, 'last_view_update'):
            self.last_view_update = {}
        current_time = time.time()
        if key not in self.last_view_update or current_time - self.last_view_update[key] > 0.1:
            # 背景画像の選択
            ref_img_key = self.dict_buttongroup["refimg"].checkedButton().text()
            pri_image = self.dict_im[key][ref_img_key].copy()

            # コントラスト調整
            pri_image = self.adjustChannelContrast(key, pri_image)

            # RGB画像の作成
            rgb_image = self.createRGBImage(key, pri_image)

            # QImageに変換
            height, width = rgb_image.shape[:2]
            qimage = QImage(rgb_image.data, width, height, width * 3, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(qimage)

            # ROIを描画(選択していないものも含む)
            self.drawAllROIs(key, pixmap)

            # シーンをクリアして新しい画像を追加
            self.dict_scene[key].clear()
            self.dict_scene[key].addPixmap(pixmap)

            # ビューを更新
            self.dict_view[key].setScene(self.dict_scene[key])
            self.dict_view[key].fitInView(self.dict_scene[key].sceneRect(), Qt.KeepAspectRatio)

            # 選択したROIの情報を表示
            self.displayROIStat(key)
            self.last_view_update[key] = current_time
        else:
            return  # 更新をスキップ

    def adjustChannelContrast(self, key, image):
        min_val = self.dict_slider[f"value_{key}_min"].value()
        max_val = self.dict_slider[f"value_{key}_max"].value()
        return np.clip(((image - min_val) / (max_val - min_val) * 255), 0, 255).astype(np.uint8)

    def createRGBImage(self, key, img):
        rgb_image = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
        if self.dict_checkbox[key].isChecked():
            rgb_image[:,:,1] = img  # 緑チャンネルにpri_imageを設定
        return rgb_image
    
    # ROIの描画
    def drawAllROIs(self, key, pixmap):
        # ROIの座標をまとめて計算
        all_roi_coords = [(roi_id, roi_stat["xpix"], roi_stat["ypix"]) 
                          for roi_id, roi_stat in self.dict_Fall[key]["stat"].items() 
                          if self.shouldDisplayROI(key, roi_id)]

        # 一括描画
        painter = QPainter(pixmap)
        painter.setRenderHint(QPainter.Antialiasing)
        opacity = self.dict_slider["opacity_allroi"].value()
        for roi_id, xpix, ypix in all_roi_coords:
            color = self.dict_roiColors[roi_id]
            fill_color = QColor(*color)
            fill_color.setAlpha(opacity)
            painter.setPen(QPen(fill_color, 1))
            for x, y in zip(xpix, ypix):
                painter.drawPoint(int(x), int(y))

        self.highlightSelectedROI(key, painter)
        painter.end()

    # 表示するROIの選別 Neuronだけ表示したりするなど
    def shouldDisplayROI(self, key, roi_id):
        selected_display = self.dict_buttongroup["roidisp"].checkedButton().text()

        if selected_display == "None":
            return False

        if selected_display != "All ROI":
            for col_name, col_info in self.dict_tablecol.items():
                if col_info['type'] == 'radio' and col_name == selected_display:
                    radio_button = self.dict_table[key].cellWidget(roi_id, col_info['order'])
                    if not (radio_button and radio_button.isChecked()):
                        return False
                    break

        # statの値で表示を選別
        if self.dict_checkbox["roidisp_stat"].isChecked():
            roi_stat = self.dict_Fall[key]["stat"][roi_id]
            npix_text = self.dict_lineedit["roidisp_npix"].text().strip()
            compact_text = self.dict_lineedit["roidisp_compact"].text().strip()

            if npix_text and roi_stat["npix"].size > 0:
                try:
                    if roi_stat["npix"][0] < float(npix_text):
                        return False
                except ValueError:
                    pass

            if compact_text and roi_stat["compact"].size > 0:
                try:
                    if roi_stat["compact"][0] > float(compact_text):
                        return False
                except ValueError:
                    pass

        return True

    # 選択したROIをハイライトして描画
    def highlightSelectedROI(self, key, painter):
        if self.dict_table_selectedRow[key] is not None:
            selected_roi_id = self.dict_table_selectedRow[key]
            selected_roi_stat = self.dict_Fall[key]["stat"][selected_roi_id]
            xpix = selected_roi_stat["xpix"]
            ypix = selected_roi_stat["ypix"]

            # 選択されたROIの色を取得
            color = self.dict_roiColors[selected_roi_id]

            # 選択されたROIの透明度を取得
            opacity = self.dict_slider["opacity_selectedroi"].value()

            # 塗りつぶしの色（選択された透明度を適用）
            highlight_color = QColor(*color)
            highlight_color.setAlpha(opacity)

            # 選択されたROIを描画
            for x, y in zip(xpix, ypix):
                painter.setPen(QPen(highlight_color, 1))
                painter.drawPoint(int(x), int(y))
                
    def onViewClicked(self, event, key):
        # ビュー内のクリック位置を取得
        scene_pos = self.dict_view[key].mapToScene(event.pos())
        clicked_x, clicked_y = scene_pos.x(), scene_pos.y()

        # 最も近いROIを見つける
        closest_roi_id = self.findClosestROI(key, clicked_x, clicked_y)

        if closest_roi_id is not None:
            # テーブル内の対応する行の Cell ID セル（1列目）を選択
            self.dict_table[key].setCurrentCell(closest_roi_id, 0)

            # テーブルにフォーカスを設定し、選択したセルが見えるようにスクロール
            self.dict_table[key].setFocus()
            self.dict_table[key].scrollToItem(self.dict_table[key].item(closest_roi_id, 0))

    # クリックした位置に最も近いROIを探す
    def findClosestROI(self, key, x, y):
        min_distance = float('inf')
        closest_roi_id = None

        for roi_id, roi_stat in self.dict_Fall[key]["stat"].items():
            # スキップ条件をチェック
            if self.shouldSkipROI(key, roi_id):
                continue

            med_x, med_y = roi_stat["med"]
            distance = ((x - med_x) ** 2 + (y - med_y) ** 2) ** 0.5
            if distance < min_distance:
                min_distance = distance
                closest_roi_id = roi_id

        return closest_roi_id

    # checkboxを参照して特定のROIはスキップする
    def shouldSkipROI(self, key, roi_id):
        # ラジオボタンタイプのカラムをチェック
        for col_name, col_info in self.dict_tablecol.items():
            skip_checkbox = self.dict_checkbox.get(f"skip_{col_name}")
            if col_info['type'] == 'radio':
                radio_button = self.dict_table[key].cellWidget(roi_id, col_info['order'])
                if radio_button and radio_button.isChecked():
                    if skip_checkbox and skip_checkbox.isChecked():
                        return True

        # チェックボックスタイプのカラムをチェック
        for col_name, col_info in self.dict_tablecol.items():
            skip_checkbox = self.dict_checkbox.get(f"skip_{col_name}")
            if col_info['type'] == 'checkbox':
                checkbox_item = self.dict_table[key].item(roi_id, col_info['order'])
                if checkbox_item and checkbox_item.checkState() == Qt.Checked:
                    if skip_checkbox and skip_checkbox.isChecked():
                        return True

        return False

    def selectROI(self, key, roi_id):
        # テーブル内の対応する行を選択
        self.dict_table[key].selectRow(roi_id)
        self.dict_table_selectedRow[key] = roi_id

        # ビューを更新
        self.updateView(key)

        # プロットを更新
        self.updatePlot(key)

        # ROIの統計情報を表示
        self.displayROIStat(key)
    
    # 表示するROIの種類切り替え
    def onROIDispButtonGroupsChanged(self, key="pri", label=None, button_id=None):
        self.updateView(key)

    # pri, sec画像表示ON/OFFの切り替え
    def toggleChannelVisibility(self, key="pri", state=None, channel=None):
        self.updateView(key)
        
    # refimg buttongroupの表示画像切り替え
    def onRefImgButtonGroupChanged(self, key="pri", label=None, button_id=None):
        self.updateView(key)
    
    
    """
    Canvas Widget Function
    """
    # プロットの更新
    def updatePlot(self, key):
        if self.dict_table_selectedRow[key] is not None:
            roi_id = self.dict_table_selectedRow[key]
            data = self.getRoiData(key, roi_id)
            time = self.getTimeAxis(key)
            
            # plot_rangeが初期化されていない場合、初期化する
            if key not in self.plot_range or self.plot_range[key][1] is None:
                self.plot_range[key] = [0, time[-1]]
            # plot_rangeが有効な値であることを確認
            if self.plot_range[key][0] is None:
                self.plot_range[key][0] = 0
            if self.plot_range[key][1] is None:
                self.plot_range[key][1] = time[-1]

            ax_top = self.dict_ax[f"{key}_top"]
            ax_middle = self.dict_ax[f"{key}_middle"]
            ax_bottom = self.dict_ax[f"{key}_bottom"]

            self.updateTopAxis(key, data, ax_top, time, roi_id)
            self.updateMiddleAxis(key, data, ax_middle, time, roi_id)
            self.updateBottomAxis(key, data, ax_bottom, time, roi_id, self.dict_eventfile[key])

            # 一括更新
            self.dict_figure[key].canvas.draw_idle()
            self.dict_figure[key].canvas.flush_events()

    # 指定されたROIのデータを取得
    def getRoiData(self, key, roi_id):
        # 効率化用のキャッシュ
        if not hasattr(self, 'roi_data_cache'):
            self.roi_data_cache = {}
        if (key, roi_id) not in self.roi_data_cache:
            self.roi_data_cache[(key, roi_id)] = {
                'F': self.dict_Fall[key]["F"][roi_id],
                'Fneu': self.dict_Fall[key]["Fneu"][roi_id],
                'spks': self.dict_Fall[key]["spks"][roi_id]
           }
        return self.roi_data_cache[(key, roi_id)]

    # 時間軸を生成
    def getTimeAxis(self, key):
        # 再計算の回避
        if not hasattr(self, 'time_axis_cache'):
            self.time_axis_cache = {}
        if key not in self.time_axis_cache:
            fs = self.dict_Fall[key]["ops"]["fs"].flatten()[0]
            self.time_axis_cache[key] = np.arange(len(self.dict_Fall[key]["F"][0])) / fs
        return self.time_axis_cache[key]

    # top axisの更新
    def updateTopAxis(self, key, data, ax, time, roi_id):
        ax.clear()
        colors = {'F': 'cyan', 'Fneu': 'red', 'spks': 'gray'}
        for name, values in data.items():
            ax.plot(time, values, color=colors[name], label=name, linewidth=0.5)

        # イベントデータのプロット
        if self.dict_eventfile[key] is not None and len(self.dict_eventfile[key]) > 0:
            if self.dict_checkbox["plot_eventfile_trace"].isChecked():
                event_data = self.scaleEventData(self.dict_eventfile[key], data['F'])
                ax.plot(time, event_data, color='green', label='Event', linewidth=0.5)

        ax.set_xlim(self.plot_range[key])
        ax.set_xlabel('Time (s)')
        ax.set_title(f'ROI {roi_id} Traces (Zoomed)')

    # middle axisの更新
    def updateMiddleAxis(self, key, data, ax, time, roi_id):
        ax.clear()
        colors = {'F': 'cyan', 'Fneu': 'red', 'spks': 'gray'}
        for name, values in data.items():
            ax.plot(time, values, color=colors[name], label=name, linewidth=0.5)

        # イベントデータのプロット
        if self.dict_eventfile[key] is not None and len(self.dict_eventfile[key]) > 0:
            if self.dict_checkbox["plot_eventfile_trace"].isChecked():
                event_data = self.scaleEventData(self.dict_eventfile[key], data['F'])
                ax.plot(time, event_data, color='green', label='Event', linewidth=0.5)

        ax.set_xlim(0, time[-1])
        ax.set_xlabel('Time (s)')
        ax.set_title(f'ROI {roi_id} Traces (Full)')

        # 紫色の四角形を追加
        rect = plt.Rectangle((self.plot_range[key][0], ax.get_ylim()[0]),
                             self.plot_range[key][1] - self.plot_range[key][0],
                             ax.get_ylim()[1] - ax.get_ylim()[0],
                             fill=False, ec='purple', lw=2)
        ax.add_patch(rect)
        
    # bottom axisの更新
    def updateBottomAxis(self, key, data, ax, time, roi_id, dict_eventfile):
        ax.clear()
        if dict_eventfile is None:
            # 全ROIの平均F、Fneu、spksをプロット
            colors = {'F': 'cyan', 'Fneu': 'red', 'spks': 'gray'}
            for trace_type, color in colors.items():
                mean_trace = np.mean(self.dict_Fall[key][trace_type], axis=0)
                ax.plot(time, mean_trace, color=color, linewidth=0.5, label=trace_type)
            ax.set_title('Average Trace')
        else:
            # イベントファイルが0から1に変化する瞬間のインデックスを取得
            event_indices = np.where(np.diff(dict_eventfile, prepend=0) == 1)[0]
            # プロット範囲を取得
            plot_range_str = self.dict_lineedit["eventfile_align_plot_range"].text().strip('()').split(',')
            b, a = map(int, plot_range_str)
            fs = self.dict_Fall[key]["ops"]["fs"].flatten()[0]
            frames_before = int(b * fs)
            frames_after = int(a * fs)
            # プロット用の時間配列を作成
            event_time = np.arange(-frames_before, frames_after) / fs
            expected_length = frames_before + frames_after
            
            # F, eventfileをプロット
            all_traces = []
            all_event_traces = []
            event_data = self.scaleEventData(dict_eventfile, data['F'])

            for idx in event_indices:
                start = max(0, idx - frames_before)
                end = min(idx + frames_after, len(self.dict_Fall[key]["F"][0]))

                trace = data["F"][start:end]
                event_trace = event_data[start:end]

                if len(trace) == expected_length and len(event_trace) == expected_length:
                    ax.plot(event_time, trace, linewidth=0.2, zorder=1, alpha=0.5)
                    ax.plot(event_time, event_trace, linewidth=0.2, zorder=2, color='green', alpha=0.5)
                    all_traces.append(trace)
                    all_event_traces.append(event_trace)

            # 平均トレースを計算して赤色でプロット
            if all_traces:
                mean_trace = np.mean(all_traces, axis=0)
                ax.plot(event_time, mean_trace, color='red', linewidth=1, label='Mean', zorder=3)

            # 相関係数の計算
            r = np.corrcoef(np.array(all_traces).flatten(), np.array(all_event_traces).flatten())[0, 1]
            ax.set_title(f'ROI {roi_id} Event Aligned Trace (r = {r:.3f})')

        ax.set_xlabel('Time (s)')

    # 上部軸のスクロールイベントハンドラ
    def onScrollTopAxis(self, event, key):
        if event.inaxes == self.dict_ax[f"{key}_top"]:
            factor = 1.5 if event.button == 'up' else 1/1.5
            self.zoomPlotRange(key, factor, event.xdata)
            self.updatePlot(key)

    # 上部軸のマウスプレスイベントハンドラ
    def onPressTopAxis(self, event, key):
        if event.inaxes == self.dict_ax[f"{key}_top"]:
            self.dragging = True
            self.last_x = event.xdata

    # 上部軸のマウスリリースイベントハンドラ
    def onReleaseTopAxis(self, event, key):
        self.dragging = False

    # 上部軸のマウス移動イベントハンドラ
    def onMotionTopAxis(self, event, key, drag_sensitivity=1.5):
        if self.dragging and event.inaxes == self.dict_ax[f"{key}_top"]:
            if event.xdata is None:
                return
            dx = (event.xdata - self.last_x) * drag_sensitivity
            self.last_x = event.xdata
            self.panPlotRange(key, dx)

            # 更新頻度を制限
            current_time = time.time()
            if not hasattr(self, 'last_update_time') or current_time - self.last_update_time > 0.05:
                self.updatePlot(key)
                self.last_update_time = current_time

    # 中部軸のクリックイベントハンドラ
    def onClickMiddleAxis(self, event, key):
        if event.inaxes == self.dict_ax[f"{key}_middle"]:
            clicked_x = event.xdata
            self.centerPlotRange(key, clicked_x)
            self.updatePlot(key)

    # プロット範囲のズーム
    def zoomPlotRange(self, key, factor, center):
        current_range = self.plot_range[key][1] - self.plot_range[key][0]
        new_range = current_range / factor
        max_range = self.getTimeAxis(key)[-1]
        min_range = float(self.dict_lineedit["minPlotRange"].text())

        if min_range <= new_range <= max_range:
            self.plot_range[key] = [
                center - new_range / 2,
                center + new_range / 2
            ]
        elif new_range > max_range:
            self.plot_range[key] = [0, max_range]
        else:
            self.plot_range[key] = [
                center - min_range / 2,
                center + min_range / 2
            ]

        self.clampPlotRange(key)

    # プロット範囲のパン
    def panPlotRange(self, key, dx):
        self.plot_range[key] = [
            self.plot_range[key][0] - dx,
            self.plot_range[key][1] - dx
        ]
        self.clampPlotRange(key)

    # プロット範囲の中心を設定
    def centerPlotRange(self, key, center):
        current_range = self.plot_range[key][1] - self.plot_range[key][0]
        self.plot_range[key] = [
            center - current_range / 2,
            center + current_range / 2
        ]
        self.clampPlotRange(key)

    # プロット範囲を有効な範囲内に制限
    def clampPlotRange(self, key):
        max_range = self.getTimeAxis(key)[-1]
        min_range = float(self.dict_lineedit["minPlotRange"].text())
        current_range = self.plot_range[key][1] - self.plot_range[key][0]

        if current_range < min_range:
            center = sum(self.plot_range[key]) / 2
            self.plot_range[key] = [center - min_range/2, center + min_range/2]
        elif current_range > max_range:
            self.plot_range[key] = [0, max_range]

        if self.plot_range[key][0] < 0:
            self.plot_range[key] = [0, current_range]
        elif self.plot_range[key][1] > max_range:
            self.plot_range[key] = [max_range - current_range, max_range]
    
    # イベントデータをスケーリングする関数
    def scaleEventData(self, event_data, original_data):
        if len(event_data) == 0 or len(original_data) == 0:
            return event_data
        event_max = np.max(event_data)
        original_max = np.max(original_data)
        if event_max == 0:
            return event_data
        # データ型を明示的にfloat64に変換
        return (event_data.astype(np.float64) * original_max) / event_max
    
    """
    Label Widget Function
    """
    # 選択したROIのstatを表示
    def displayROIStat(self, key):
        if self.dict_table_selectedRow[key] is not None:
            roi_id = self.dict_table_selectedRow[key]
            roi_stat = self.dict_Fall[key]["stat"][roi_id]

            # 表示する統計情報
            stat_keys = ["npix", "radius", "aspect_ratio", "compact", "footprint", "skew", "std"]

            for stat_key in stat_keys:
                if stat_key in roi_stat:
                    value = roi_stat[stat_key]
                    if isinstance(value, np.ndarray):
                        value = value[0]  # 配列の場合は最初の要素を使用
                    self.dict_label[f"display_{stat_key}"].setText(f"{stat_key}: {value:.2f}")
                else:
                    self.dict_label[f"display_{stat_key}"].setText(f"{stat_key}: N/A")
              
    # それぞれcelltypeのROI数をカウントして表示
    def displayROINumber(self, key):
        # ラジオタイプのカラムを抽出
        radio_columns = [col_name for col_name, col_info in self.dict_tablecol.items() if col_info['type'] == 'radio']

        # 各カテゴリのROI数をカウント
        roi_counts = {col_name: 0 for col_name in radio_columns}
        total_rois = 0

        for row in range(self.dict_table[key].rowCount()):
            total_rois += 1
            for col_name in radio_columns:
                col_index = self.dict_tablecol[col_name]['order']
                radio_button = self.dict_table[key].cellWidget(row, col_index)
                if radio_button and radio_button.isChecked():
                    roi_counts[col_name] += 1
                    break

        # テキストの作成
        text = ""
        for col_name in radio_columns:
            text += f"{col_name}: {roi_counts[col_name]}, "
        text += f"All: {total_rois}"

        # ラベルの更新
        self.dict_label["ROInumber"].setText(text)
    
    """
    Button Widget Function
    """
        
    def exitApp(self):
        self.close()
        
    # ほかのクラスを呼び出す関数
    """
    Other Class Function
    """
    def runHoloStimResponse(self):
        self.class_HoloStimResponse = HoloStimResponse(self, self.dict_Fall["pri"]["F"])
        self.class_HoloStimResponse.show()
        
    
    
    

    
    
    
    
    
    
    
    
# ホログラム刺激に対する細胞の応答率を求めるクラス
class HoloStimResponse(QMainWindow, WidgetManager):
    def __init__(self, parent=None, F=None):
        QMainWindow.__init__(self, parent)
        WidgetManager.__init__(self)
        self.setWindowTitle("HoloStim Response Analysis")  # ウィンドウタイトルを設定
        self.parent = parent  # Suite2pROICheckGUIへの参照を保存
        self.F = F  # F データを保存
        self.trial_data = None
        self.iti_data = None
        self.stim_duration_data = None
        self.response_threshold = 30  # デフォルト値

        self.last_analyzed_roi = None
        self.plot_range = [0, None]
        self.initUI()
        
        # タイマーをセットアップ
        self.update_timer = QTimer(self)
        self.update_timer.timeout.connect(self.checkAndUpdateAnalysis)
        self.update_timer.start(100)  # 100ミリ秒ごとにチェック

    def initUI(self):
        central_widget = QWidget()
        self.setCentralWidget(central_widget)

        self.mainLayout = QGridLayout(central_widget)

        # コントロールレイアウト
        Layout_control = QHBoxLayout()

        # ファイル読み込みボタン
        Layout_control.addWidget(self.makeWidgetButton("load_holostim", 'Load Holo Stim Files', self.loadHoloStimFiles))

        # 閾値設定
        Layout_threshold = QHBoxLayout()
        Layout_threshold.addWidget(self.makeWidgetLabel("threshold", 'Response Threshold (frames):'))
        Layout_threshold.addWidget(self.makeWidgetLineEdit("threshold", str(self.response_threshold)))
        Layout_control.addLayout(Layout_threshold)

        # プロット用のキャンバス
        self.figure, (self.dict_ax["top"], self.dict_ax["bottom"]) = plt.subplots(2, 1, figsize=(10, 10))
        self.figure.subplots_adjust(top=0.95, bottom=0.1, right=0.9, left=0.1, hspace=0.4)
        self.canvas = FigureCanvas(self.figure)

        self.mainLayout.addWidget(self.canvas, 0, 0)
        self.mainLayout.addLayout(Layout_control, 1, 0)

        self.setMinimumSize(600, 400)  # 最小幅600px、最小高さ400px
        self.canvas.setMinimumSize(400, 300)  # キャンバスの最小サイズも設定

    def updatePlot(self, roi_id):
        if self.F is not None and self.iti_data is not None and self.stim_duration_data is not None:
            data = self.getRoiData(roi_id)
            time = self.getTimeAxis()
            
            if self.plot_range[1] is None:
                self.plot_range[1] = time[-1]

            self.parent.updateBottomAxis("pri", data, self.dict_ax["top"], time, roi_id, self.iti_data)
            self.parent.updateBottomAxis("pri", data, self.dict_ax["bottom"], time, roi_id, self.stim_duration_data)
            if hasattr(self, "response_ratio"):
                title_top = self.dict_ax["top"].get_title()
                title_top += f"\n ROI {roi_id} ItiDuration, response = {self.response_ratio:.2f}"
                self.dict_ax["top"].set_title(title_top)
                title_bottom = self.dict_ax["bottom"].get_title()
                title_bottom += f"\n ROI {roi_id} StimDuration, response = {self.response_ratio:.2f}"
                self.dict_ax["bottom"].set_title(title_bottom)
            else:
                # 条件に合わない場合、プロットをクリア
                self.dict_ax["top"].clear()
                self.dict_ax["bottom"].clear()
            
            self.canvas.draw()

    def getRoiData(self, roi_id):
        return {
            'F': self.F[roi_id],
        }

    def getTimeAxis(self):
        return self.parent.getTimeAxis("pri")

    def loadHoloStimFiles(self):
        options = QFileDialog.Options()
        files, _ = QFileDialog.getOpenFileNames(self, "Load Holo Stim Files", "", "Numpy Files (*.npy)", options=options)
        if files:
            for file in files:
                if 'TrialDuration' in file:
                    self.trial_data = np.load(file)
                elif 'ItiDuration' in file:
                    self.iti_data = np.load(file)
                elif 'StimDuration' in file:
                    self.stim_duration_data = np.load(file)
        
        if self.trial_data is not None and self.iti_data is not None and self.stim_duration_data is not None:
            print("All files loaded successfully")
            self.checkAndUpdateAnalysis()

    def checkAndUpdateAnalysis(self):
        current_roi = self.getCurrentROI()
        if current_roi is not None and current_roi != self.last_analyzed_roi:
            if self.trial_data is not None and self.iti_data is not None and self.stim_duration_data is not None:
                self.analyzeHoloStimResponse(current_roi)
                self.updatePlot(current_roi)
                self.last_analyzed_roi = current_roi

    def getCurrentROI(self):
        return self.parent.dict_table_selectedRow.get("pri")
    
    def analyzeHoloStimResponse(self, roi_id):
        self.response_threshold = int(self.dict_lineedit["threshold"].text())

        trial_periods = self.get_consecutive_periods(self.trial_data)
        response_ratios = []

        for start, end in trial_periods:
            try:
                f_data = self.F[roi_id][start:end]
                iti_data = self.iti_data[start:end]
                stim_data = self.stim_duration_data[start:end]

                iti_threshold = self.calculate_iti_threshold(f_data, iti_data)
                response = self.detect_response(f_data, stim_data, iti_threshold)

                response_ratios.append(1 if response else 0)
            except Exception as e:
                continue
#             print(iti_threshold)

#         print(response_ratios)
        self.response_ratio = np.mean(response_ratios)

    def get_consecutive_periods(self, data):
        diff = np.diff(np.concatenate(([0], data, [0])))
        starts = np.where(diff == 1)[0]
        ends = np.where(diff == -1)[0]
        return list(zip(starts, ends))

    def calculate_iti_threshold(self, f_data, iti_data):
        iti_f_data = f_data[iti_data == 1]  # ITI期間中のF値データを抽出
        return np.mean(iti_f_data) + 2 * np.std(iti_f_data)

    def detect_response(self, f_data, stim_data, threshold):
        stim_periods = self.get_consecutive_periods(stim_data)
        for stim_start, stim_end in stim_periods:
            stim_f_data = f_data[stim_start:stim_end]
            above_threshold = stim_f_data > threshold
            if np.any(np.convolve(above_threshold, np.ones(self.response_threshold), mode='valid') == self.response_threshold):
                return True
        return False
    
    
if __name__ == "__main__":
    app = QApplication(sys.argv) if QApplication.instance() is None else QApplication.instance()
#     font = setGUIFont(app)
    gui = Suite2pROICheckGUI()
    gui.show()
    sys.exit(app.exec_())

c:\Users\sofut\Machine_Learning\Suite2pROITracking\optic


SystemExit: 0

c:\Users\sofut\anaconda3\envs\itkelastix\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
dir_parent

['c:\\Users\\sofut\\anaconda3\\envs\\itkelastix\\python38.zip',
 'c:\\Users\\sofut\\anaconda3\\envs\\itkelastix\\DLLs',
 'c:\\Users\\sofut\\anaconda3\\envs\\itkelastix\\lib',
 'c:\\Users\\sofut\\anaconda3\\envs\\itkelastix',
 '',
 'c:\\Users\\sofut\\anaconda3\\envs\\itkelastix\\lib\\site-packages',
 'c:\\Users\\sofut\\anaconda3\\envs\\itkelastix\\lib\\site-packages\\win32',
 'c:\\Users\\sofut\\anaconda3\\envs\\itkelastix\\lib\\site-packages\\win32\\lib',
 'c:\\Users\\sofut\\anaconda3\\envs\\itkelastix\\lib\\site-packages\\Pythonwin']

In [5]:
parent_dir

'c:\\Users\\sofut\\Machine_Learning\\Suite2pROITracking\\optic'

In [1]:
from optic.preprocessing import *

In [2]:
convertMatToDictFall

<function optic.preprocessing.preprocessing_fall.convertMatToDictFall(Fall)>